# functions: print_result

[print_result](../api/functions.rst#nornir_utils.plugins.functions.print_result) is a function that takes a result and prints it. For instance, imagine the following code:

In [1]:
from nornir import InitNornir

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

result = nr.run(task=dummy_task)

Now we could print the result easily with the `print_result` function: 

In [2]:
from nornir_utils.plugins.functions import print_result

print_result(result)

dummy_task**********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^ END dummy_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv dummy_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

## Printing specific data

If the task returns different pieces of information you can also select which ones to print. For instance:

In [3]:
from nornir.core.task import Result

def task_with_data(task):
    return Result(host=task.host, sw_char="Jar Jar Binks", food="hawaiian pizza", OS="windows")


fav_result = nr.run(task=task_with_data)

print_result(fav_result, vars=["sw_char", "food"]) # print only these vars

task_with_data******************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv task_with_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
Jar Jar Binks
hawaiian pizza
^^^^ END task_with_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv ta

## Severity

You can also ask `print_result` to print the results only if the severity is equal or superior to the one specified. This is particularly useful if your script is very large and you don't care about intermediate results. For instance:

In [4]:
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)

`print_result` will only print `INFO` severity and above by default so if we print the result we should only see the result of the parent task:

In [5]:
print_result(complex_result)

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev4.group_2 ** changed : False **********************************************
vvvv complex_task *

If you wanted to print all the tasks for debugging purposes you could specify it with:

In [6]:
print_result(complex_result, severity_level=logging.DEBUG)

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
---- dummy_task ** changed : False --------------------------------------------- DEBUG
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of

## No_errors

If some `Result` objects have exceptions, then you can exclude them from the output of the `print_result` function:

In [7]:
def task_with_exception(task):
    return Result(host=task.host, result="Something went wrong", exception=Exception())


def task_without_exception(task):
    return "All is fine"

def task(task):
    task.run(task=task_with_exception)
    task.run(task=task_without_exception)
    return "What's happening?"

    
result_with_exception = nr.run(task=task)

print_result(result_with_exception)

task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_with_exception ** changed : False ------------------------------------ INFO
Something went wrong
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [8]:
print_result(result_with_exception, no_errors=True)

task****************************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- task_without_exception ** changed : False --------------------------------- INFO
All is fine
^^^^ END task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev3.group_2 ** changed : False **********************************************
vvvv task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
What's happening?
---- t

## Print_host

[write_result](../api/functions.rst#nornir_utils.plugins.functions.write_result) and [write_results](../api/functions.rst#nornir_utils.plugins.functions.write_results) functions are based on `print_result` function with `file` argument, so the `print_host` argument can be used by them, for example, when there is already hostname in the filename and you don't want to duplicate it.

`print_host` argument can also help you, if you don't need to print hostnames for `MultiResult` or `Result` object.

In [9]:
def complex_task(task):
    task.run(task=dummy_task)
    task.run(task=dummy_task)
    task.run(task=dummy_task)
    task.run(task=dummy_task)
    task.run(task=dummy_task)
    return "I did a lot of things!!!"

complex_result = nr.run(task=complex_task)

For `MultiResult` object:

In [10]:
print_result(complex_result['dev1.group_1'])

vvvv dev1.group_1: complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [11]:
print_result(complex_result['dev1.group_1'], print_host=False)

vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


For `Result` object:

In [12]:
print_result(complex_result['dev1.group_1'][1])

---- dev1.group_1: dummy_task ** changed : False ------------------------------- INFO
hi!!!


In [13]:
print_result(complex_result['dev1.group_1'][1], print_host=False)

---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!


## File

The `file` argument must be an object with a write(string) method; if it is not present or `None`, `sys.stdout` will be used. The `file` argument of the `print_result` function uses the same `file` argument of the `print` function:

In [14]:
with open("out_files/print_result.txt", "w+") as f:
    print_result(complex_result, file=f)

In [15]:
with open("out_files/print_result.txt") as f:
    print(f.read())

complex_task********************************************************************
* dev1.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
---- dummy_task ** changed : False --------------------------------------------- INFO
hi!!!
^^^^ END complex_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* dev2.group_1 ** changed : False **********************************************
vvvv complex_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of thin